In [ ]:
import requests
import re

def classify_sentiment_ollama(content):
    # 设置系统消息（如果需要）
    system_message = """
    你是一个游戏评论的方面级情感分析助手。请对用户的评论内容，从以下六个方面提取“意见术语”以及“情感类型”，并以标准JSON格式输出。

六个方面为：
1. 剧情
2. 画面
3. 玩法
4. 创意
5. 对男性的看法（如对男性角色、男性形象、对现实中男性的态度等）
6. 对女性的看法（如对女性角色、女性形象、对现实中女性的态度等）

分析规则如下：

1. 对每个方面，判断评论是否涉及该方面内容；
2. 如果涉及该方面，请从评论中抽取一个或多个最能反映该方面评价的关键词、短语，作为“意见术语”，如“新颖”“拖沓”“普通”“单调”“刻板印象”“正面形象”等；
3. 并根据语境，将该方面的情感归类到如下9个情感类型之一（只选最合适的一个）：
   - 惊喜（感到出乎意料的好）
   - 赞美（直接表达称赞、夸奖）
   - 开心（表达愉快、满意的情绪）
   - 乐观（对后续有积极期待）
   - 担忧（对未来有负面预期
   - 讽刺（带有嘲讽、挖苦或反话意味）
   - 生气（表达愤怒、不满）
   - 失望（期待未达成、落差）
   - 中立（客观描述，没有明确情感）

4. 如果评论没有涉及某个方面，"意见术语"和"情感类型"都填写“没有提及”。
5. 输出时必须使用如下标准JSON格式，字段和顺序不能改动，所有文本用英文半角双引号。

6. 不要输出除JSON外的任何内容，不要有解释或说明。

【标准示例1】

评论：
剧情节奏拖沓，画面很普通，玩法让人失望，感觉并没有多少创意，男性角色形象很工具人，女主又是刻板印象，看得很不舒服。

输出：
{
  "剧情": {"意见术语": "节奏拖沓", "情感类型": "失望"},
  "画面": {"意见术语": "很普通", "情感类型": "中立"},
  "玩法": {"意见术语": "让人失望", "情感类型": "失望"},
  "创意": {"意见术语": "没有多少创意", "情感类型": "失望"},
  "对男性的看法": {"意见术语": "工具人", "情感类型": "生气"},
  "对女性的看法": {"意见术语": "刻板印象", "情感类型": "讽刺"}
}

【标准示例2】

评论：
剧情很紧凑，画面好看，玩法很有趣，创意很棒，男性女性角色塑造都很立体。

输出：
{
  "剧情": {"意见术语": "很紧凑", "情感类型": "赞美"},
  "画面": {"意见术语": "画面好看", "情感类型": "开心"},
  "玩法": {"意见术语": "很有趣", "情感类型": "开心"},
  "创意": {"意见术语": "很棒", "情感类型": "惊喜"},
  "对男性的看法": {"意见术语": "角色塑造立体", "情感类型": "赞美"},
  "对女性的看法": {"意见术语": "角色塑造立体", "情感类型": "赞美"}
}

【标准示例3】

评论：
觉得这个游戏设定很大胆，剧情没什么特别，玩法有点无聊，其他方面没关注。

输出：
{
  "剧情": {"意见术语": "没什么特别", "情感类型": "中立"},
  "画面": {"意见术语": "没有提及", "情感类型": "没有提及"},
  "玩法": {"意见术语": "有点无聊", "情感类型": "失望"},
  "创意": {"意见术语": "设定大胆", "情感类型": "惊喜"},
  "对男性的看法": {"意见术语": "没有提及", "情感类型": "没有提及"},
  "对女性的看法": {"意见术语": "没有提及", "情感类型": "没有提及"}
}

请严格按照上述规则、词表和格式输出分析结果。
    """

    # 构建请求的payload，传入用户评论
    payload = {
        "model": "Qwen3:8b",  # 本地部署的模型名称
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": content}
        ],
        "stream": False,  # 禁用流式响应
        "temperature": 0 # 设置温度
    }
    
    try:
        # 向本地API发送POST请求
        response = requests.post("http://localhost:11434/api/chat", json=payload)
        
        # 确保请求成功
        response.raise_for_status()
        
        # 解析JSON响应
        resp_json = response.json()
        
        # 提取返回的情感分析结果
        label = resp_json['message']['content'].strip()
        label = re.sub(r"<think>.*?</think>", "", label, flags=re.DOTALL).strip()
        
        return label

    except requests.exceptions.RequestException as e:
        print(f"API调用失败, 错误: {e}")
        return "API调用失败"
    except KeyError:
        print("响应格式错误: 没有返回预期的内容")
        return "API响应格式错误"

# 示例：调用分析函数
content = "游戏本身：这东西还是推荐入手，知识点总结很有用，后续帮助产生个人思考与实际应用的空间非常大，但游戏剧情反而差劲，满分五分只能有三分半左右，选项过于极端化，未考虑有介入因素导致的额外选择，要么完全被真心引导舍弃利益，要么完全理性考虑自己利益最大化，这导致了剧本自由度不高，配不上模拟二字，但这个问题不大，都懂，如果有理性思考之后的两全平衡的选项，即使最后因为剧本导致会失败会更好，没有这个让我的代入感非常差劲，这东西因人而异，我本人光速结局感觉白玩了，因为想两全但又没有机会，完全没办法单视角玩这个游戏，只能去双视角玩，和导演编剧斗地主了属于是，代入军师看月半🐱如何发挥，我在旁边做指导作用，哈哈哈哈哈哈，只能这么玩了，这么玩确实有意思。关于性别问题：没有女生版本确实有说法，如果有女生版本对比绝对不会输，如果有还输了更好，更能撕掉非人的面目，现状是男性势弱，从均衡来讲的话这游戏绝对是应时代需求产生的佳作，不能抛开时代背景简单的去审判这个游戏如何污名化女性，这种行为就是抛开事实，这个游戏会让男生集体右转，因为一味前行者已经证明了猴急的是SB，保守理性的筛选，才能在现实社会这个恶臭的泥潭里淘出金子（这是大家被迫的手段，不应该是目的），异化的人群无法意识到自己的错误，并且认为这是被迫谋生的事。最后，人只能拿利益换到利益，拿真心换到真心，而只考虑利益最大化，并且是其中佼佼者的人，早应该被吊在路灯上了"
result = classify_sentiment_ollama(content)

# 打印情感分析结果
print(f"情感分析结果: {result}")


情感分析结果: {
  "剧情": {"意见术语": "差劲", "情感类型": "失望"},
  "画面": {"意见术语": "没有提及", "情感类型": "没有提及"},
  "玩法": {"意见术语": "选项极端化", "情感类型": "失望"},
  "创意": {"意见术语": "知识点总结", "情感类型": "赞美"},
  "对男性的看法": {"意见术语": "集体右转", "情感类型": "赞美"},
  "对女性的看法": {"意见术语": "污名化女性", "情感类型": "讽刺"}
}


In [ ]:
import pandas as pd
import time
import json
import os

# 读取数据
df = pd.read_csv('steam_reviews.csv')
df_cn = df[df['language'] == 'schinese'].reset_index(drop=True)

output_path = 'steam_reviews_sentiment.jsonl'

df_cn = df_cn[df_cn['review'].astype(str).str.len() > 15].reset_index(drop=True)

# 获取已处理的行数（如果有历史结果）
processed_idx = set()
if os.path.exists(output_path):
    with open(output_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                obj = json.loads(line)
                processed_idx.add(obj['idx'])
            except Exception as e:
                continue

print(f"已检测到{len(processed_idx)}条已处理评论，将自动跳过")

with open(output_path, 'a', encoding='utf-8') as fout:
    for idx, row in df_cn.iterrows():
        if idx in processed_idx:
            continue
        review_text = row['review']
        try:
            result = classify_sentiment_ollama(review_text)
        except Exception as e:
            result = "API调用失败"
            print(f"第{idx}条评论分析失败：{e}")
        item = {
            "idx": idx,
            "review": review_text,
            "情感分析结果": result
        }
        fout.write(json.dumps(item, ensure_ascii=False) + '\n')
        print(f'已完成第{idx+1}/{len(df_cn)}条')
        time.sleep(0.3)


已检测到0条已处理评论，将自动跳过
已完成第1/11663条
已完成第2/11663条
已完成第3/11663条
已完成第4/11663条
已完成第5/11663条
已完成第6/11663条
已完成第7/11663条
已完成第8/11663条
已完成第9/11663条
已完成第10/11663条
已完成第11/11663条
已完成第12/11663条
已完成第13/11663条
已完成第14/11663条
已完成第15/11663条
已完成第16/11663条
已完成第17/11663条
已完成第18/11663条
已完成第19/11663条
已完成第20/11663条
已完成第21/11663条
已完成第22/11663条
已完成第23/11663条
已完成第24/11663条
已完成第25/11663条
已完成第26/11663条
已完成第27/11663条
已完成第28/11663条
已完成第29/11663条
已完成第30/11663条
已完成第31/11663条
已完成第32/11663条
已完成第33/11663条
已完成第34/11663条
已完成第35/11663条
已完成第36/11663条
已完成第37/11663条
已完成第38/11663条
已完成第39/11663条
已完成第40/11663条
已完成第41/11663条
已完成第42/11663条
已完成第43/11663条
已完成第44/11663条
已完成第45/11663条
已完成第46/11663条
已完成第47/11663条
已完成第48/11663条
已完成第49/11663条
已完成第50/11663条
已完成第51/11663条
已完成第52/11663条
已完成第53/11663条
已完成第54/11663条
已完成第55/11663条
已完成第56/11663条
已完成第57/11663条
已完成第58/11663条
已完成第59/11663条
已完成第60/11663条
已完成第61/11663条
已完成第62/11663条
已完成第63/11663条
已完成第64/11663条
已完成第65/11663条
已完成第66/11663条
已完成第67/11663条
已完成第68/11663条
已完成第69/11663条
已完成第70/11663条
已完成第71/1166

In [ ]:
import os
import pandas as pd
import json
from openai import OpenAI
import time

client = OpenAI(
    api_key="sk-0a956abd729440fe9d6b3ac66893c884",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 阿里Qwen兼容OpenAI接口
)

# 方面级情感分析的system_message（用你之前整理的详细版本即可！）
system_message = """
    你是一个游戏评论的方面级情感分析助手。请对用户的评论内容，从以下六个方面提取“意见术语”以及“情感类型”，并以标准JSON格式输出。

六个方面为：
1. 剧情
2. 画面
3. 玩法
4. 创意
5. 对男性的看法（如对男性角色、男性形象、对现实中男性的态度等）
6. 对女性的看法（如对女性角色、女性形象、对现实中女性的态度等）

分析规则如下：

1. 对每个方面，判断评论是否涉及该方面内容；
2. 如果涉及该方面，请从评论中抽取一个或多个最能反映该方面评价的关键词、短语，作为“意见术语”，如“新颖”“拖沓”“普通”“单调”“刻板印象”“正面形象”等；
3. 并根据语境，将该方面的情感归类到如下9个情感类型之一（只选最合适的一个）：
   - 惊喜（感到出乎意料的好）
   - 赞美（直接表达称赞、夸奖）
   - 开心（表达愉快、满意的情绪）
   - 乐观（对后续有积极期待）
   - 担忧（对未来有负面预期
   - 讽刺（带有嘲讽、挖苦或反话意味）
   - 生气（表达愤怒、不满）
   - 失望（期待未达成、落差）
   - 中立（客观描述，没有明确情感）

4. 如果评论没有涉及某个方面，"意见术语"和"情感类型"都填写“没有提及”。
5. 输出时必须使用如下标准JSON格式，字段和顺序不能改动，所有文本用英文半角双引号。

6. 不要输出除JSON外的任何内容，不要有解释或说明。

【标准示例1】

评论：
剧情节奏拖沓，画面很普通，玩法让人失望，感觉并没有多少创意，男性角色形象很工具人，女主又是刻板印象，看得很不舒服。

输出：
{
  "剧情": {"意见术语": "节奏拖沓", "情感类型": "失望"},
  "画面": {"意见术语": "很普通", "情感类型": "中立"},
  "玩法": {"意见术语": "让人失望", "情感类型": "失望"},
  "创意": {"意见术语": "没有多少创意", "情感类型": "失望"},
  "对男性的看法": {"意见术语": "工具人", "情感类型": "生气"},
  "对女性的看法": {"意见术语": "刻板印象", "情感类型": "讽刺"}
}

【标准示例2】

评论：
剧情很紧凑，画面好看，玩法很有趣，创意很棒，男性女性角色塑造都很立体。

输出：
{
  "剧情": {"意见术语": "很紧凑", "情感类型": "赞美"},
  "画面": {"意见术语": "画面好看", "情感类型": "开心"},
  "玩法": {"意见术语": "很有趣", "情感类型": "开心"},
  "创意": {"意见术语": "很棒", "情感类型": "惊喜"},
  "对男性的看法": {"意见术语": "角色塑造立体", "情感类型": "赞美"},
  "对女性的看法": {"意见术语": "角色塑造立体", "情感类型": "赞美"}
}

    """

# 输入数据
df = pd.read_csv('steam_reviews.csv')
df_cn = df[df['language'] == 'schinese'].reset_index(drop=True)
df_cn = df_cn[df_cn['review'].astype(str).str.len() > 15].reset_index(drop=True)

output_path = "steam_reviews_sentiment.jsonl"

# 读取已处理部分，支持断点续算
processed_idx = set()
if os.path.exists(output_path):
    with open(output_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
                processed_idx.add(obj['idx'])
            except Exception as e:
                continue

print(f"已检测到{len(processed_idx)}条已处理评论，将自动跳过")

# API方面级情感分析
def classify_sentiment_ali(review_text):
    try:
        completion = client.chat.completions.create(
            model="deepseek-r1-distill-qwen-7b",   # 按你的实际模型填写
            messages=[
                {'role': 'system', 'content': system_message},
                {'role': 'user', 'content': review_text}
            ]
        )
        response = json.loads(completion.model_dump_json())
        label = response['choices'][0]['message']['content'].strip()
        # 确认返回是标准JSON
        try:
            label_json = json.loads(label)
            return label_json
        except:
            return label
    except Exception as e:
        print(f"API error at: {review_text}, error: {e}")
        return "API调用失败"

# 主循环
import sys
with open(output_path, 'a', encoding='utf-8') as fout:
    for idx, row in df_cn.iterrows():
        if idx in processed_idx:
            continue
        review_text = row['review']
        try:
            result = classify_sentiment_ali(review_text)
        except Exception as e:
            result = "API调用失败"
            print(f"第{idx}条评论分析失败：{e}")
        item = {
            "idx": idx,
            "review": review_text,
            "情感分析结果": result
        }
        fout.write(json.dumps(item, ensure_ascii=False) + '\n')
        fout.flush()
        os.fsync(fout.fileno())
        print(f'已完成第{idx+1}/{len(df_cn)}条')
        time.sleep(0.2)  # 可根据API QPS调整


已检测到3468条已处理评论，将自动跳过
已完成第3469/11663条
已完成第3470/11663条
已完成第3471/11663条
已完成第3472/11663条
已完成第3473/11663条
已完成第3474/11663条
已完成第3475/11663条
已完成第3476/11663条
已完成第3477/11663条
已完成第3478/11663条
已完成第3479/11663条
已完成第3480/11663条
已完成第3481/11663条
已完成第3482/11663条
已完成第3483/11663条
已完成第3484/11663条
已完成第3485/11663条
已完成第3486/11663条
已完成第3487/11663条
已完成第3488/11663条
已完成第3489/11663条
已完成第3490/11663条
已完成第3491/11663条
已完成第3492/11663条
已完成第3493/11663条
已完成第3494/11663条
已完成第3495/11663条
已完成第3496/11663条
已完成第3497/11663条
已完成第3498/11663条
已完成第3499/11663条
已完成第3500/11663条
已完成第3501/11663条
已完成第3502/11663条
已完成第3503/11663条
已完成第3504/11663条
已完成第3505/11663条
已完成第3506/11663条
已完成第3507/11663条
已完成第3508/11663条
已完成第3509/11663条
已完成第3510/11663条
已完成第3511/11663条
已完成第3512/11663条
已完成第3513/11663条
已完成第3514/11663条
已完成第3515/11663条
已完成第3516/11663条
已完成第3517/11663条
已完成第3518/11663条
已完成第3519/11663条
已完成第3520/11663条
已完成第3521/11663条
已完成第3522/11663条
已完成第3523/11663条
已完成第3524/11663条
已完成第3525/11663条
已完成第3526/11663条
已完成第3527/11663条
已完成第3528/11663条
已完成第3529/11663条
已完成

In [ ]:
import os
import pandas as pd
import json
import time
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="")

# 方面级情感分析的system_message（用你之前整理的详细版本即可！）
system_message = """
    你是一个游戏评论的方面级情感分析助手。请对用户的评论内容，从以下六个方面提取“意见术语”以及“情感类型”，并以标准JSON格式输出。

六个方面为：
1. 剧情
2. 画面
3. 玩法
4. 创意
5. 对男性的看法（如对男性角色、男性形象、对现实中男性的态度等）
6. 对女性的看法（如对女性角色、女性形象、对现实中女性的态度等）

分析规则如下：

1. 对每个方面，判断评论是否涉及该方面内容；
2. 如果涉及该方面，请从评论中抽取一个或多个最能反映该方面评价的关键词、短语，作为“意见术语”，如“新颖”“拖沓”“普通”“单调”“刻板印象”“正面形象”等；
3. 并根据语境，将该方面的情感归类到如下9个情感类型之一（只选最合适的一个）：
   - 惊喜（感到出乎意料的好）
   - 赞美（直接表达称赞、夸奖）
   - 开心（表达愉快、满意的情绪）
   - 乐观（对后续有积极期待）
   - 担忧（对未来有负面预期
   - 讽刺（带有嘲讽、挖苦或反话意味）
   - 生气（表达愤怒、不满）
   - 失望（期待未达成、落差）
   - 中立（客观描述，没有明确情感）

4. 如果评论没有涉及某个方面，"意见术语"和"情感类型"都填写“没有提及”。
5. 输出时必须使用如下标准JSON格式，字段和顺序不能改动，所有文本用英文半角双引号。

6. 不要输出除JSON外的任何内容，不要有解释或说明。

【标准示例1】

评论：
剧情节奏拖沓，画面很普通，玩法让人失望，感觉并没有多少创意，男性角色形象很工具人，女主又是刻板印象，看得很不舒服。

输出：
{
  "剧情": {"意见术语": "节奏拖沓", "情感类型": "失望"},
  "画面": {"意见术语": "很普通", "情感类型": "中立"},
  "玩法": {"意见术语": "让人失望", "情感类型": "失望"},
  "创意": {"意见术语": "没有多少创意", "情感类型": "失望"},
  "对男性的看法": {"意见术语": "工具人", "情感类型": "生气"},
  "对女性的看法": {"意见术语": "刻板印象", "情感类型": "讽刺"}
}

【标准示例2】

评论：
剧情很紧凑，画面好看，玩法很有趣，创意很棒，男性女性角色塑造都很立体。

输出：
{
  "剧情": {"意见术语": "很紧凑", "情感类型": "赞美"},
  "画面": {"意见术语": "画面好看", "情感类型": "开心"},
  "玩法": {"意见术语": "很有趣", "情感类型": "开心"},
  "创意": {"意见术语": "很棒", "情感类型": "惊喜"},
  "对男性的看法": {"意见术语": "角色塑造立体", "情感类型": "赞美"},
  "对女性的看法": {"意见术语": "角色塑造立体", "情感类型": "赞美"}
}

    """

# 输入数据
df = pd.read_csv('steam_reviews.csv')
df_cn = df[df['language'] == 'schinese'].reset_index(drop=True)
df_cn = df_cn[df_cn['review'].astype(str).str.len() > 15].reset_index(drop=True)

output_path = "steam_reviews_sentiment.jsonl"

# 读取已处理部分，支持断点续算
processed_idx = set()
if os.path.exists(output_path):
    with open(output_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
                processed_idx.add(obj['idx'])
            except Exception as e:
                continue

print(f"已检测到{len(processed_idx)}条已处理评论，将自动跳过")

# API方面级情感分析
def classify_sentiment_ali(review_text):
    try:
        response = client.chat.completions.create(
            model="glm-4-flash-250414",   # 按你的实际模型填写
            messages=[
                {'role': 'system', 'content': system_message},
                {'role': 'user', 'content': review_text}
            ],
            top_p= 0.11,
            temperature= 0,
            max_tokens=5050,
            stream=False
        )
        label = response.choices[0].message.content
        # 确认返回是标准JSON
        try:
            label_json = json.loads(label)
            return label_json
        except:
            return label
    except Exception as e:
        print(f"API error at: {review_text}, error: {e}")
        return "API调用失败"

# 主循环
import sys
with open(output_path, 'a', encoding='utf-8') as fout:
    for idx, row in df_cn.iterrows():
        if idx in processed_idx:
            continue
        review_text = row['review']
        try:
            result = classify_sentiment_ali(review_text)
        except Exception as e:
            result = "API调用失败"
            print(f"第{idx}条评论分析失败：{e}")
        item = {
            "idx": idx,
            "review": review_text,
            "情感分析结果": result
        }
        fout.write(json.dumps(item, ensure_ascii=False) + '\n')
        fout.flush()
        os.fsync(fout.fileno())
        print(f'已完成第{idx+1}/{len(df_cn)}条')
        time.sleep(0.2)  # 可根据API QPS调整


已检测到4603条已处理评论，将自动跳过
已完成第4604/11663条
已完成第4605/11663条
已完成第4606/11663条
已完成第4607/11663条
已完成第4608/11663条
已完成第4609/11663条
已完成第4610/11663条
已完成第4611/11663条
已完成第4612/11663条
已完成第4613/11663条
已完成第4614/11663条
已完成第4615/11663条
已完成第4616/11663条
已完成第4617/11663条
已完成第4618/11663条
已完成第4619/11663条
已完成第4620/11663条
已完成第4621/11663条
已完成第4622/11663条
已完成第4623/11663条
已完成第4624/11663条
已完成第4625/11663条
已完成第4626/11663条
已完成第4627/11663条
已完成第4628/11663条
已完成第4629/11663条
已完成第4630/11663条
已完成第4631/11663条
已完成第4632/11663条
已完成第4633/11663条
已完成第4634/11663条
已完成第4635/11663条
已完成第4636/11663条
已完成第4637/11663条
已完成第4638/11663条
已完成第4639/11663条
已完成第4640/11663条
已完成第4641/11663条
已完成第4642/11663条
已完成第4643/11663条
已完成第4644/11663条
已完成第4645/11663条
已完成第4646/11663条
已完成第4647/11663条
已完成第4648/11663条
已完成第4649/11663条
已完成第4650/11663条
已完成第4651/11663条
已完成第4652/11663条
已完成第4653/11663条
已完成第4654/11663条
已完成第4655/11663条
已完成第4656/11663条
已完成第4657/11663条
已完成第4658/11663条
已完成第4659/11663条
已完成第4660/11663条
已完成第4661/11663条
已完成第4662/11663条
已完成第4663/11663条
已完成第4664/11663条
已完成

In [6]:
import json
import pandas as pd
import re

# 文件路径
jsonl_path = 'steam_reviews_sentiment.jsonl'
csv_path = 'sentiment_aspects.csv'

data = []

bad_lines = 0

with open(jsonl_path, 'r', encoding='utf-8') as f:
    for idx, line in enumerate(f):
        line = line.strip()
        if not line:
            continue  # 跳过空行
        try:
            row = json.loads(line)
        except Exception as e:
            print(f"第{idx+1}行不是合法json，已跳过：", e)
            bad_lines += 1
            continue

        review = row.get("review", "")
        result = {
            "idx": row.get("idx", ""),
            "评论": review
        }

        aspect_json = row.get("情感分析结果")
        aspect_dict = {}

        if isinstance(aspect_json, str):
            # 1. 去除markdown块和多余空白
            clean_str = aspect_json.strip()
            clean_str = re.sub(r"^```json", "", clean_str)
            clean_str = re.sub(r"^```", "", clean_str)
            clean_str = re.sub(r"```$", "", clean_str)
            clean_str = clean_str.strip()
            # 2. 尽量只提取第一个合法json
            match = re.search(r'\{[\s\S]+\}', clean_str)
            if match:
                clean_str = match.group(0)
            # 3. json解析，错误就跳过
            try:
                aspect_dict = json.loads(clean_str)
            except Exception as e:
                print(f"第{idx+1}行情感字段解析失败：", e)
                bad_lines += 1
                aspect_dict = {}
        elif isinstance(aspect_json, dict):
            aspect_dict = aspect_json

        # 六大方面
        for aspect in ["剧情", "画面", "玩法", "创意", "对男性的看法", "对女性的看法"]:
            v = aspect_dict.get(aspect, {})
            if isinstance(v, dict):
                result[f"{aspect}_意见术语"] = v.get("意见术语", "")
                result[f"{aspect}_情感类型"] = v.get("情感类型", "")
            else:
                result[f"{aspect}_意见术语"] = ""
                result[f"{aspect}_情感类型"] = ""
        data.append(result)

print(f"共处理{idx+1}行，其中异常/坏行：{bad_lines} 行")

df = pd.DataFrame(data)
df.to_csv(csv_path, index=False, encoding='utf-8-sig')
print("转换完成，已保存为：", csv_path)

第124行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第205行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第304行情感字段解析失败： Expecting ':' delimiter: line 4 column 15 (char 102)
第319行情感字段解析失败： Expecting ',' delimiter: line 7 column 26 (char 240)
第355行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第502行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第670行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第696行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第707行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第766行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第790行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第844行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第1013行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第1105行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第1144行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第1276行情感字段解析失败： Expecting value: line 1 column 1 (char 0)
第1291行情感字段解析失败： Expecting ':' delimiter: line 4 column 14 (cha